<a href="https://colab.research.google.com/github/2303a51852/AIML1852/blob/main/Assinment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from queue import Queue
from copy import deepcopy
import numpy as np
import time

# Adjacency list graph definition
graph = {0: [1, 3], 1: [0, 2, 3], 2: [4, 1, 5], 3: [4, 0, 1], 4: [2, 3, 5], 5: [4, 2], 6: []}

# Printing the adjacency list
print("The adjacency List representing the graph is:")
print(graph)

# BFS traversal
def bfs(graph, source):
    Q = Queue()
    visited_vertices = set()
    Q.put(source)
    visited_vertices.add(source)
    while not Q.empty():
        vertex = Q.get()
        print(vertex, end="-->")
        for u in graph[vertex]:
            if u not in visited_vertices:
                Q.put(u)
                visited_vertices.add(u)

# Running BFS
print("\nBFS traversal of graph with source 0 is:")
bfs(graph, 0)
print("\n")

# DFS traversal
graph1 = {
    'A': ['B', 'S'],
    'B': ['A'],
    'C': ['D', 'E', 'F', 'S'],
    'D': ['C'],
    'E': ['C', 'H'],
    'F': ['C', 'G'],
    'G': ['F', 'S'],
    'H': ['E', 'G'],
    'S': ['A', 'C', 'G']
}

def dfs(graph, node, visited):
    if node not in visited:
        visited.append(node)
        for k in graph[node]:
            dfs(graph, k, visited)
    return visited

# Running DFS
visited = dfs(graph1, 'D', [])
print(visited)

# Best solution finding
def bestsolution(state):
    bestsol = np.array([], int).reshape(-1, 9)
    count = len(state) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, state[count]['puzzle'], 0)
        count = state[count]['parent']
    return bestsol.reshape(-1, 3, 3)

# All check function
def all_check(setlist, checkarray):
    for it in setlist:
        if np.array_equal(it, checkarray):
            return 1
    return 0

# Misplaced tiles function
def misplaced_tiles(puzzle, goal):
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0

# Coordinates function
def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos

# Evaluation of misplaced tiles
def evaluvate_misplaced(puzzle, goal):
    steps = np.array([('up', [0, 1, 2], -3), ('down', [6, 7, 8], 3), ('left', [0, 3, 6], -1), ('right', [2, 5, 8], 1)],
                     dtype=[('move', str, 1), ('position', list), ('head', int)])
    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]

    costg = coordinates(goal)
    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(puzzle), costg)
    state = np.array([(puzzle, parent, gn, hn)], dtstate)

    dtpriority = [('position', int), ('fn', int)]
    priority = np.array([(0, hn)], dtpriority)

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        priority = np.delete(priority, 0, 0)
        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)

        blank = int(np.where(puzzle == 0)[0])
        gn += 1
        start_time = time.time()
        for s in steps:
            if blank not in s['position']:
                openstates = deepcopy(puzzle)
                openstates[blank], openstates[blank + s['head']] = openstates[blank + s['head']], openstates[blank]

                if not np.any([np.array_equal(state_el['puzzle'], openstates) for state_el in state]):
                    end_time = time.time()
                    if (end_time - start_time) > 2:
                        print("The 8 puzzle is unsolvable\n")
                        return state, len(priority)
                    hn = misplaced_tiles(coordinates(openstates), costg)
                    q = np.array([(openstates, position, gn, hn)], dtstate)
                    state = np.append(state, q, 0)
                    fn = gn + hn
                    q = np.array([(len(state) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)

                    if np.array_equal(openstates, goal):
                        print("The 8 puzzle is solvable\n")
                        return state, len(priority)

    return state, len(priority)

# Puzzle initialization
puzzle = [2, 8, 3, 7, 1, 4, 0, 6, 5]
goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]

# Evaluate misplaced tiles and find the best solution
state, visited = evaluvate_misplaced(puzzle, goal)
bestpath = bestsolution(state)
print(str(bestpath).replace('[', ' ').replace(']', ''))
totalmoves = len(bestpath) - 1
print(f'\nSteps to reach goal: {totalmoves}')
visit = len(state) - visited
print(f'Total nodes visited: {visit} \n')


The adjacency List representing the graph is:
{0: [1, 3], 1: [0, 2, 3], 2: [4, 1, 5], 3: [4, 0, 1], 4: [2, 3, 5], 5: [4, 2], 6: []}

BFS traversal of graph with source 0 is:
0-->1-->3-->2-->4-->5-->

['D', 'C', 'E', 'H', 'G', 'F', 'S', 'A', 'B']
The 8 puzzle is solvable

   2 8 3
   7 1 4
   0 6 5

   2 8 3
   0 1 4
   7 6 5

   2 8 3
   1 0 4
   7 6 5

   2 0 3
   1 8 4
   7 6 5

   0 2 3
   1 8 4
   7 6 5

   1 2 3
   0 8 4
   7 6 5

   1 2 3
   8 0 4
   7 6 5

Steps to reach goal: 6
Total nodes visited: 11 



<ipython-input-2-e8cab9a26cbc>:106: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blank = int(np.where(puzzle == 0)[0])
